# Sachsen

Every federal state is represented by its own input directory and is processed into a NUTS level 2 directory containing a sub-folder for each discharge location. These folder names are derived from NUTS and reflect the CAMELS id. The NUTS level 2 code for Sachsen is `DED`.

To pre-process the data, you need to write (at least) two functions. One should extract all metadata and condense it into a single `pandas.DataFrame`. This is used to build the folder structure and derive the ids.
The second function has to take an id, as provided by the state authorities, called `provider_id` and return a `pandas.DataFrame` with the transformed data. The dataframe needs the three columns `['date', 'q' | 'w', 'flag']`.

For easier and unified output handling, the `camelsp` package contains a context object called `Bundesland`. It takes a number of names and abbreviations to identify the correct federal state and returns an object that holds helper and save functions.

The context saves files as needed and can easily be changed to save files with different strategies, ie. fill missing data with NaN, merge data into a single file, create files for each variable or pack everything together into a netcdf.

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import geopandas as gpd
from pandas.errors import ParserError
import os
from glob import glob
from pprint import pprint
from tqdm import tqdm
from typing import Union, Dict
from datetime import datetime as dt
from dateparser import parse
import warnings

from camelsp import Bundesland, Station

The context can also be instantiated as any regular Python class, ie. to load only the default input data path, that we will user later.

In [2]:
# the context also makes the input path available, if camelsp was install locally
BASE = Bundesland('Sachsen').input_path
BASE

'/home/alexd/Projekte/CAMELS/Github/camelsp/input_data/SN_Sachsen'

## Parse data

We do not have a Metadata file, but one Excel file for each station. Thus we need to parse each metadata individually and collect.

In [3]:
files = glob(os.path.join(BASE, '*.xlsx'))
print(f"Found {len(files)} files.")

Found 282 files.


Test for the first file:

In [4]:
with warnings.catch_warnings():
    df = pd.read_excel(files[32], skiprows=2, decimal=',')


print(f"Pegelkennziffer: {df.Pegelkennziffer.unique()}")
print(f"Pegelname:       {df.Pegelname.unique()}")
print(f"Gewaesser:       {df.Gewaesser.unique()}")
print(f"Beeinflussung:   {df.Beeinflussung.unique()}")
print(f"Datum type:      {df.Datum.dtype}")

df

/home/alexd/miniconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Pegelkennziffer: [660193]
Pegelname:       ['Podrosche 3']
Gewaesser:       ['Lausitzer Neiße']
Beeinflussung:   [nan 'b' 'e' 'R' 'R, e' 'R, T' 'b, e' 'K']
Datum type:      datetime64[ns]


,Pegelkennziffer,Pegelname,Gewaesser,Datum,Wasserstand (W) cm,Durchfluss (Q) m³/s,Beeinflussung
0,660193,Podrosche 3,Lausitzer Neiße,1984-11-01,NaN,9.36,NaN
1,660193,Podrosche 3,Lausitzer Neiße,1984-11-02,NaN,9.36,NaN
2,660193,Podrosche 3,Lausitzer Neiße,1984-11-03,NaN,9.36,NaN
3,660193,Podrosche 3,Lausitzer Neiße,1984-11-04,NaN,9.36,NaN
4,660193,Podrosche 3,Lausitzer Neiße,1984-11-05,NaN,8.78,NaN
...,...,...,...,...,...,...,...
12900,660193,Podrosche 3,Lausitzer Neiße,2020-12-27,76.0,12.50,NaN
12901,660193,Podrosche 3,Lausitzer Neiße,2020-12-28,75.0,12.10,NaN
12902,660193,Podrosche 3,Lausitzer Neiße,2020-12-29,71.0,11.10,NaN
12903,660193,Podrosche 3,Lausitzer Neiße,2020-12-30,70.0,10.50,NaN


One question, what does Beeinflussung actually mean here? Ignoring it for now.

Go for each file and extract metadata and the two data columns

In [5]:
np.unique(df[df.columns[1]].values)

array(['Podrosche 3'], dtype=object)

In [6]:
# create result container
meta = []
q = []
w = []

with warnings.catch_warnings(record=True) as warns:
    for filename in tqdm(files):
        # read
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            df = pd.read_excel(filename, skiprows=2, decimal=',')

        # extract data columns
        q_df = df[['Datum', 'Durchfluss (Q) m³/s']].copy()
        q_df.columns = ['date', 'q']
        q_df['flag'] = np.NaN
        
        w_df = df[['Datum', 'Wasserstand (W) cm']].copy()
        w_df.columns = ['date', 'w']
        w_df['flag'] = np.NaN
        
        # append
        q.append(q_df)
        w.append(w_df)

        # metadata - get first three columns
        m = dict()
        for i in range(3):
            # these columns need to be unique
            if np.unique(df[df.columns[i]].values).size > 1:
                warnings.warn(f"Column {df.columns[i]} of file {filename} is expected to be unique")
                m = None
                break
            else:
                # add metadata
                m[df.columns[i]] = str(df.iloc[0, i])
        
        # add other stuff
        if m is not None:     
            m['unit_q'] = 'm³/s'
            m['unit_w'] = 'cm'
        meta.append(m)
    
print(f"Parsed {len(meta)} files")

100%|██████████| 282/282 [03:12<00:00,  1.47it/s]

Parsed 282 files


### Create metadta

this should be straightforward

In [7]:
metadata = pd.DataFrame(meta)
metadata

,Pegelkennziffer,Pegelname,Gewaesser,unit_q,unit_w
0,551431,Dippoldiswalde 3,Werkgraben,m³/s,cm
1,550190,Porschdorf 1,Lachsbach,m³/s,cm
2,663090,Tauchritz,Pließnitz,m³/s,cm
3,564201,Niedermülsen 1,Mülsenbach,m³/s,cm
4,564200,Niedermülsen,Mülsenbach,m³/s,cm
...,...,...,...,...,...
277,563745,Johanngeorgenstadt 4,Schwarzwasser,m³/s,cm
278,583223,Reichwalde 3,Schwarzer Schöps,m³/s,cm
279,560301,Nemt 1,Mühlbach,m³/s,cm
280,583100,Löbau Stadion,Löbauer Wasser,m³/s,cm


There is more metadata in the provided Pegel shapefile, including critical information such as location.

In [8]:
gdf_meta = gpd.read_file(os.path.join(BASE, 'Pegel_SHP/PEGEL.shp'))

# drop column geometry
gdf_meta.drop(columns='geometry', inplace=True)

# provider_id as int
gdf_meta['MSTNR'] = gdf_meta['MSTNR'].astype(int)
metadata['Pegelkennziffer'] = metadata['Pegelkennziffer'].astype(int)

# concat metadata and gdf_meta
metadata = metadata.merge(gdf_meta, left_on='Pegelkennziffer', right_on='MSTNR', how='left')

# make Pegelkennziffer str again
metadata['Pegelkennziffer'] = metadata['Pegelkennziffer'].astype(str)

In [9]:
metadata[metadata['Pegelkennziffer'] == '662082']

,Pegelkennziffer,Pegelname,Gewaesser,unit_q,unit_w,OBJECTID,MSTNR,NAME,ART,ORDNUNG,...,AS3,AS4,OSTWERT,NORDWERT,EDVW_BIS,EDVQ_BIS,GEBKZ,GEWAESSER,FLUSSGEB,DATENSTAND
115,662082,Neuschönau 2,Lasur,m³/s,cm,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
id_column = 'Pegelkennziffer'

### Finally run

Now, the Q and W data can be extracted. The cool thing is, that all the id creation, data creation, merging and the mapping from our ids to the original ids and files is done by the context. This is helpful, as we less likely screw something up.

In [11]:
with Bundesland('Sachsen') as bl:
    # save the metadata
    bl.save_raw_metadata(metadata, id_column, overwrite=True)

    # for reference, call the nuts-mapping as table
    nuts_map = bl.nuts_table
    print(nuts_map.head())

    # go for each    
    for m, q_df, w_df in tqdm(zip(meta, q, w), total=len(meta)):
        
        if m is not None:
            # get the provider id
            provider_id = str(m[id_column])
            bl.save_timeseries(q_df, provider_id)
            bl.save_timeseries(w_df, provider_id)

    # check if there were warnings (there are warnings)
    if len(warns) > 0:
        log_path = bl.save_warnings(warns)
        print(f"There were warnings during the processing. The log can be found at: {log_path}")


    nuts_id provider_id                              path
0  DED10000      551431  ./DED/DED10000/DED10000_data.csv
1  DED10010      550190  ./DED/DED10010/DED10010_data.csv
2  DED10020      663090  ./DED/DED10020/DED10020_data.csv
3  DED10030      564201  ./DED/DED10030/DED10030_data.csv
4  DED10040      564200  ./DED/DED10040/DED10040_data.csv


100%|██████████| 282/282 [00:13<00:00, 21.43it/s]


## Add EZG from provider to all Stations where available

The provided shapefile does not contain the station id but only station name.  
We try to get the station id by matching station name.

In [32]:
gdf_ezg = gpd.read_file(os.path.join(BASE, '../Shapes/Sachsen_Shapes/EZG_SACHS/EZG_SACHS.shp'))

for station_name in gdf_ezg['Pegelname'].values:
    # get station id from station name and river name
    id = meta[meta['gauge_name'] == station_name].camels_id.values
    #print(id)
    # if id was found, init station
    # if id.size > 0:
    #     s = Station(id[0])

    #     # get catchment geometry for id
    #     catchment = gdf_ezg[(gdf_ezg['NAME'] == station_name) & (gdf_ezg['GEWäSSER'] == river_name)].iloc[[0]]

    #     # save catchment geometry
    #     s.save_catchment_geometry(catchment, datasource='federal_agency_ezg')


gdf_ezg

,Flussgeb,Area,Pegelname,geometry
0,"Weiße Elster, Eger",11.864755,Bobenneukirchen,"POLYGON ((293270.844 5584840.910, 293281.362 5..."
1,"Weiße Elster, Eger",19.938839,Neuensalz,"POLYGON ((303911.707 5598875.465, 303935.022 5..."
2,Mulde,10.492721,Eibenstock2,"POLYGON ((328604.565 5597009.363, 328606.516 5..."
3,Mulde,29.792558,Markersbach1,"POLYGON ((351709.111 5597752.695, 351734.066 5..."
4,Mulde,10.338390,NeunzehnhainIIZP1,"POLYGON ((368749.795 5618588.938, 368743.447 5..."
5,Mulde,21.697303,Wolfersgruen,"POLYGON ((318316.075 5611301.865, 318329.855 5..."
6,Mulde,91.098487,Tannenberg,"POLYGON ((352365.486 5611370.764, 352407.377 5..."
7,Elbe,2.411241,Gottleuba2,"POLYGON ((424553.556 5630946.868, 424578.707 5..."
8,Mulde,37.805546,Wolfsgrund,"POLYGON ((389073.791 5626155.289, 389102.234 5..."
9,Elbe,6.269273,Baerenfels,"POLYGON ((406664.783 5626971.576, 406688.514 5..."


Those are only large catchments in Sachsen, there are no matches with Pegelname and gauge_name in our metadata...  
Can't do anything with that.